In [550]:
import requests
import pandas as pd
import json
from dateutil import parser


In [551]:
API_KEY = "a2faf0838c2aacce44be67cea8b40c06-48d69bac26199a4d5850294881134c34"
ACCOUNT_ID = "101-004-21509763-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [552]:
session = requests.Session()

In [553]:
session.headers.update({
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
})

In [554]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [555]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [556]:
response = session.get(url, params=None, data=None, headers=None)

In [557]:
response.status_code

200

In [558]:
data = response.json()

In [559]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [560]:
instruments_list = data["instruments"]

In [561]:
len(instruments_list)

123

In [562]:
instruments_list[1].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [563]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [564]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i }

In [565]:
instruments_dict['AU200_AUD']

{'name': 'AU200_AUD',
 'type': 'CFD',
 'displayName': 'Australia 200',
 'pipLocation': 0,
 'displayPrecision': 1,
 'tradeUnitsPrecision': 1,
 'marginRate': '0.05'}

In [566]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [567]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    

In [568]:

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
            
        final_data.append(new_dict)
    df = pd.DataFrame(final_data)
    return df


In [569]:
code, data = fetch_candles('EUR_USD', count=20, granularity="H4")

In [570]:
code

200

In [571]:
data[0]

{'complete': True,
 'volume': 30806,
 'time': '2023-10-31T13:00:00.000000000Z',
 'bid': {'o': '1.06233', 'h': '1.06251', 'l': '1.05567', 'c': '1.05668'},
 'mid': {'o': '1.06240', 'h': '1.06258', 'l': '1.05574', 'c': '1.05676'},
 'ask': {'o': '1.06248', 'h': '1.06265', 'l': '1.05582', 'c': '1.05684'}}

In [572]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']
for p in prices:
    for o in ohlc:
        print(f"{p}_{o}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [573]:
get_candles_df(data)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-10-31 13:00:00+00:00,30806,1.06240,1.06258,1.05574,1.05676,1.06233,1.06251,1.05567,1.05668,1.06248,1.06265,1.05582,1.05684
1,2023-10-31 17:00:00+00:00,11925,1.05675,1.05858,1.05660,1.05744,1.05668,1.05851,1.05652,1.05735,1.05682,1.05867,1.05668,1.05754
2,2023-10-31 21:00:00+00:00,4868,1.05743,1.05806,1.05687,1.05735,1.05709,1.05799,1.05680,1.05727,1.05777,1.05814,1.05694,1.05743
3,2023-11-01 01:00:00+00:00,8027,1.05732,1.05766,1.05660,1.05672,1.05725,1.05758,1.05652,1.05665,1.05740,1.05774,1.05668,1.05680
4,2023-11-01 05:00:00+00:00,11230,1.05672,1.05776,1.05571,1.05583,1.05665,1.05768,1.05564,1.05575,1.05679,1.05785,1.05578,1.05591
5,2023-11-01 09:00:00+00:00,17876,1.05583,1.05628,1.05369,1.05386,1.05576,1.05621,1.05361,1.05379,1.05590,1.05636,1.05377,1.05394
6,2023-11-01 13:00:00+00:00,28535,1.05385,1.05710,1.05214,1.05338,1.05377,1.05702,1.05206,1.05331,1.05393,1.05717,1.05222,1.05346
7,2023-11-01 17:00:00+00:00,34099,1.05339,1.05759,1.05166,1.05690,1.05332,1.05752,1.05158,1.05682,1.05346,1.05766,1.05174,1.05699
8,2023-11-01 21:00:00+00:00,6414,1.05704,1.06021,1.05670,1.06020,1.05654,1.06014,1.05636,1.06013,1.05754,1.06028,1.05696,1.06027
9,2023-11-02 01:00:00+00:00,7694,1.06019,1.06023,1.05933,1.05962,1.06011,1.06016,1.05926,1.05954,1.06027,1.06030,1.05940,1.05969
